### Difference Features

In [2]:
# !sudo apt update
# !sudo apt install openjdk-17-jre-headless -y
import pyspark
from pyspark import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import col, substring
from pyspark.sql.types import StructType, StructField, \
StringType, IntegerType, TimestampType, DateType, FloatType


In [3]:
conf = pyspark.SparkConf().setAll([\
            ('spark.app.name', 'Glucose_Analysis_Spark')])\
            .set('spark.sql.shuffle.partitions', '1500')
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()  

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/17 00:19:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/17 00:19:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/17 00:19:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [35]:
df = spark.read.options(header='True', inferSchema='True', delimiter=',')\
        .csv('/cephfs/data/cohort.csv')\
        .withColumnRenamed('_c0', 'NumId')

In [36]:
df.show(2)

23/05/17 00:26:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
+-----+--------------------+------+-------------------+---+------------+---------+
|NumId|              UserId|Gender|                DOB|Age|DiabetesType|Treatment|
+-----+--------------------+------+-------------------+---+------------+---------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|
+-----+--------------------+------+-------------------+---+------------+---------+
only showing top 2 rows



In [37]:
df = df.withColumn('AgeGroup', substring(df.Age.cast(StringType()), 0,1))

In [38]:
df = df.withColumnRenamed('Gender', 'Sex')

In [39]:
df.show(3)

+-----+--------------------+------+-------------------+---+------------+---------+--------+
|NumId|              UserId|   Sex|                DOB|Age|DiabetesType|Treatment|AgeGroup|
+-----+--------------------+------+-------------------+---+------------+---------+--------+
|    0|5lZPrCk6qk8L6Jw+S...|Female|1931-01-01 00:00:00| 92|    type-two|       no|       9|
|    1|9qY9mZ+GV5Kd/O/NB...|  Male|1937-01-01 00:00:00| 86|    type-two|       no|       8|
|    2|uhsyLhr4Zl6NfGbNB...|Female|1938-01-01 00:00:00| 85|    type-two|       no|       8|
+-----+--------------------+------+-------------------+---+------------+---------+--------+
only showing top 3 rows

23/05/17 00:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UserId, Gender, DOB, Age, DiabetesType, Treatment
 Schema: _c0, UserId, Gender, DOB, Age, DiabetesType, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv


In [40]:
encodedCols = ['Sex', 'Treatment', 'AgeGroup'] # not doing'DiabetesType' because all type-two
encodedLabels = []

for name in encodedCols:
    indexer = StringIndexer(inputCol=name, outputCol= name + '_Num')
    indexer_fitted = indexer.fit(df)
    encodedLabels.append([name, indexer_fitted.labels])                    

In [41]:
encodedLabels

[['Sex', ['Female', 'Male']],
 ['Treatment', ['yes-both', 'yes-long-acting', 'no', 'yes-fast-acting']],
 ['AgeGroup', ['5', '6', '7', '4', '3', '8', '9', '1']]]

In [42]:
for label in encodedLabels:
    main = label[0]
    categories = label[1]
    for cat in categories:
        cat = cat.replace('-', '_')
        if main == 'Sex':
            df = df.withColumn(main + '_' + cat, (df.Sex == cat).cast('integer'))
        elif main == 'Treatment':
            df = df.withColumn(main + '_' + cat, (df.Treatment == cat).cast('integer'))
        elif main == 'AgeGroup':
            df = df.withColumn(main + '_' + cat, (df.AgeGroup == cat).cast('integer'))
            

In [44]:
df.columns

['NumId',
 'UserId',
 'Sex',
 'DOB',
 'Age',
 'DiabetesType',
 'Treatment',
 'AgeGroup',
 'Sex_Female',
 'Sex_Male',
 'Treatment_yes_both',
 'Treatment_yes_long_acting',
 'Treatment_no',
 'Treatment_yes_fast_acting',
 'AgeGroup_5',
 'AgeGroup_6',
 'AgeGroup_7',
 'AgeGroup_4',
 'AgeGroup_3',
 'AgeGroup_8',
 'AgeGroup_9',
 'AgeGroup_1']

In [47]:
df = df.drop('UserId', 'Sex', 'DOB', 'Age', 'DiabetesType', 'Treatment', 'AgeGroup')

In [53]:
df.dtypes

[('NumId', 'int'),
 ('Sex_Female', 'int'),
 ('Sex_Male', 'int'),
 ('Treatment_yes_both', 'int'),
 ('Treatment_yes_long_acting', 'int'),
 ('Treatment_no', 'int'),
 ('Treatment_yes_fast_acting', 'int'),
 ('AgeGroup_5', 'int'),
 ('AgeGroup_6', 'int'),
 ('AgeGroup_7', 'int'),
 ('AgeGroup_4', 'int'),
 ('AgeGroup_3', 'int'),
 ('AgeGroup_8', 'int'),
 ('AgeGroup_9', 'int'),
 ('AgeGroup_1', 'int')]

In [52]:
df.write.mode('overwrite').parquet('/cephfs/data/cohort_encoded.parquet')

23/05/17 00:29:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Gender, Age, Treatment
 Schema: _c0, Gender, Age, Treatment
Expected: _c0 but found: 
CSV file: file:///cephfs/data/cohort.csv
